Report:
1. The respective columns you calc in Table 1 of Gu et al (2020)
2. The respective columns in Tables 5 and A.7 of Gu et al (2020)

You will need FF3 and FF5 porfolio breakpoiints and weights for Table 5 and A.7. Available in the course bucket

In [22]:
import numpy as np
import pandas as pd
from sklearn import metrics

In [23]:
data_dir = '/Users/anishkahc/Desktop/Docs/ML Fall 23/Data/'

openap_file = 'OpenAP_Macro.parquet.gzip'
openap = pd.read_parquet( data_dir + openap_file)

#Code from HW03
openap = openap[pd.DatetimeIndex(openap['DateYM']).year >= 2000]
openap.drop(openap.iloc[:,-10:-1], inplace = True, axis = 1)
openap.set_index(keys = ['DateYM', 'permno'], inplace = True, verify_integrity = True)
#rank
cols = openap.columns[:-1]
openap[cols] = openap.groupby('DateYM')[cols].rank(pct=True).subtract(0.5).multiply(2)


In [24]:
CMA_ret = pd.read_parquet( data_dir +'Gu-Table5/CMA_ret.parquet.gzip')
CMA_wt = pd.read_parquet( data_dir +'Gu-Table5/CMA_wt.parquet.gzip')
FF3_ret = pd.read_parquet( data_dir +'Gu-Table5/FF-3_ret.parquet.gzip')
FF3_wt = pd.read_parquet( data_dir +'Gu-Table5/FF-3_wt.parquet.gzip')
RMW_ret = pd.read_parquet( data_dir +'Gu-Table5/RMW_ret.parquet.gzip')
RMW_wt = pd.read_parquet( data_dir +'Gu-Table5/RMW_wt.parquet.gzip')
SP500_VW_ret = pd.read_parquet( data_dir +'Gu-Table5/SP500_VW_ret.parquet.gzip')
SP500_VW_wt = pd.read_parquet( data_dir +'Gu-Table5/SP500_VW_wt.parquet.gzip')
UMD_ret = pd.read_parquet( data_dir +'Gu-Table5/UMD_ret.parquet.gzip')
UMD_wt = pd.read_parquet( data_dir +'Gu-Table5/UMD_wt.parquet.gzip')

In [25]:
#missing dataset replaced by relevant period median value code from HW03
def fill_missing_values(x):
    median_value = x.median()
    return x.fillna(0 if pd.isna(median_value) else median_value)

openap = openap.groupby('DateYM').transform(fill_missing_values)
# print(openap.columns)

In [26]:
def split_val(openap):
    return model_selection.PredefinedSplit(openap['test_fold'])

In [27]:
def R2_oos(y, y_pred):
    mse = 0
    mse_benchmark = 0
    for i in range(len(y)):
        mse += (y[i] - y_pred[i])**2
        mse_benchmark += y[i]**2
    return 1 - mse/mse_benchmark

Huber loss reference: 
https://towardsdatascience.com/understanding-the-3-most-common-loss-functions-for-machine-learning-regression-23e0ef3e14d3

In [28]:
'''
def huber_loss(y_true, y_pred, delta=1.0):
    residual = y_true - y_pred
    condition = np.abs(residual) <= delta
    squared_loss = 0.5 * np.square(residual)
    linear_loss = delta * (np.abs(residual) - 0.5 * delta)
    return np.where(condition, squared_loss, linear_loss).mean()
'''

'\ndef huber_loss(y_true, y_pred, delta=1.0):\n    residual = y_true - y_pred\n    condition = np.abs(residual) <= delta\n    squared_loss = 0.5 * np.square(residual)\n    linear_loss = delta * (np.abs(residual) - 0.5 * delta)\n    return np.where(condition, squared_loss, linear_loss).mean()\n'

XGBoost Ref: https://xgboost.readthedocs.io/en/stable/tutorials/rf.html

In [29]:
import xgboost as xgb

In [30]:
#Code from class notes
def train_predict_reg (model, train_X, train_y, test_X, test_y):
    model.fit(X=train_X, y=train_y)
    
    y_train_pred = model.predict(train_X)
    y_test_pred = model.predict(test_X)
    
    r2_train = metrics.r2_score(y_true=train_y, y_pred=y_train_pred)
    r2_test = metrics.r2_score(y_true=test_y, y_pred=y_test_pred)
    
    print('Training R2:', r2_train)
    print('Test R2:', r2_test)

    return None

In [31]:
year=[]
train_r2 = []
r2_result = []


train_start = 2000
train_end = 2005
validation_end = 2008
test_end = 2009

n = 10
    
for i in range(n):
    print("Training year ", i + 2009) 
    year.append(i + 2009)

    train_val = openap[(openap.index.get_level_values('DateYM').year >= train_start) & (openap.index.get_level_values('DateYM').year <= validation_end+i)].copy()
    test = openap[(openap.index.get_level_values('DateYM').year > validation_end+i) & (openap.index.get_level_values('DateYM').year <= test_end+i)].copy()
    
    # keep 'test_fold' -1 for training sample and 0 for validation
    train_val = train_val.assign(test_fold = -1)
    train_val.loc[pd.DatetimeIndex(train_val.index.get_level_values('DateYM')).year > train_end+i, 'test_fold'] = 0
    
    # Referred to XGBoost documentation
    # Initialize XGBoost Random Forest Regressor
    xgbrf = xgb.XGBRFRegressor(n_estimators=200,
                               max_depth=10,
                               colsample_bynode=0.8,  # Equivalent to max_features in RandomForest
                               n_jobs=4,
                               random_state=3462)

    # Train the model and assess performance with 'y' as outcome.
    train_predict_reg(model=xgbrf,
                      train_X=train_val.drop(columns=['test_fold', 'retadj']),
                      train_y=train_val['retadj'],
                      test_X=test.drop(columns=['retadj']),
                      test_y=test['retadj'])

Training year  2009
Training R2: 0.2550303177105817
Test R2: -0.44527993055924386
Training year  2010
Training R2: 0.2637672228582
Test R2: -0.0700120496893859
Training year  2011
Training R2: 0.25370189682036426
Test R2: -0.03950971673940451
Training year  2012
Training R2: 0.24780437064040794
Test R2: -0.02079380560088584
Training year  2013
Training R2: 0.2386610874708841
Test R2: -0.05066994764655308
Training year  2014
Training R2: 0.23367439842879645
Test R2: -0.0833604491526867
Training year  2015
Training R2: 0.22843886660038604
Test R2: -0.00643556287058078
Training year  2016
Training R2: 0.21394812597978086
Test R2: -0.023960784033491445
Training year  2017
Training R2: 0.21840749261896264
Test R2: -0.018089111616836462
Training year  2018


Exception ignored on calling ctypes callback function: <bound method DataIter._next_wrapper of <xgboost.data.SingleBatchInternalIter object at 0x16e1e0a90>>
Traceback (most recent call last):
  File "/Applications/Anaconda/anaconda3/lib/python3.11/site-packages/xgboost/core.py", line 588, in _next_wrapper
    def _next_wrapper(self, this: None) -> int:  # pylint: disable=unused-argument

KeyboardInterrupt: 


Training R2: 0.2144111527757634
Test R2: -0.034452664727173676


### Columns for Table 5 and A.7

In [12]:
CMA_ret.head()

,jdate,port,ret
498,1963-06-30,BA,-0.024526
499,1963-06-30,BC,-0.033256
500,1963-06-30,BM,-0.007229
501,1963-06-30,SA,-0.014628
502,1963-06-30,SC,-0.010668


In [13]:
CMA_wt.head()

,jdate,permno,port,port_wt
11,1988-07-31,10001.0,SM,0.000042
12,1988-08-31,10001.0,SM,0.000044
13,1988-09-30,10001.0,SM,0.000047
14,1988-10-31,10001.0,SM,0.000044
15,1988-11-30,10001.0,SM,0.000047


In [14]:
# Data for year 2000 and after
CMA_ret = CMA_ret[pd.DatetimeIndex(CMA_ret['jdate']).year >= 2000]
CMA_wt = CMA_wt[pd.DatetimeIndex(CMA_wt['jdate']).year >= 2000]

FF3_ret = FF3_ret[pd.DatetimeIndex(FF3_ret['jdate']).year >= 2000]
FF3_wt = FF3_wt[pd.DatetimeIndex(FF3_wt['jdate']).year >= 2000]

RMW_ret = RMW_ret[pd.DatetimeIndex(RMW_ret['jdate']).year >= 2000]
RMW_wt = RMW_wt[pd.DatetimeIndex(RMW_wt['jdate']).year >= 2000]

SP500_VW_ret = SP500_VW_ret[pd.DatetimeIndex(SP500_VW_ret['jdate']).year >= 2000]
SP500_VW_wt = SP500_VW_wt[pd.DatetimeIndex(SP500_VW_wt['jdate']).year >= 2000]

UMD_ret = UMD_ret[pd.DatetimeIndex(UMD_ret['jdate']).year >= 2000]
UMD_wt = UMD_wt[pd.DatetimeIndex(UMD_wt['jdate']).year >= 2000]

In [15]:
print(CMA_ret)

          jdate port       ret
3060 2000-01-31   BA -0.057577
3061 2000-01-31   BC -0.007716
3062 2000-01-31   BM -0.043734
3063 2000-01-31   SA -0.005573
3064 2000-01-31   SC  0.004742
...         ...  ...       ...
4219 2018-05-31   SM  0.067947
4220 2018-06-30   BA  0.005457
4221 2018-06-30   BM  0.005063
4222 2018-06-30   SA  0.011524
4223 2018-06-30   SM  0.011061

[1164 rows x 3 columns]


In [16]:
# wt_ret is weighted_return
def calc_wt_ret(ret_df, wt_df):
    # merging the return and wt dataframes on 'jdate'
    merged_df = pd.merge(ret_df, wt_df, on='jdate') 
    # Calculating weighted return
    merged_df['wt_return'] = merged_df['ret'] * merged_df['port_wt']
    merged_df = merged_df.groupby('jdate')['wt_return'].sum().reset_index()
    return merged_df

In [17]:
CMA_wt_ret = calc_wt_ret(CMA_ret, CMA_wt)
FF3_wt_ret = calc_wt_ret(FF3_ret, FF3_wt)
RMW_wt_ret = calc_wt_ret(RMW_ret, RMW_wt)
SP500_wt_ret = calc_wt_ret(SP500_VW_ret, SP500_VW_wt)
UMD_wt_ret = calc_wt_ret(UMD_ret, UMD_wt)

In [18]:
UMD_wt_ret.head()
# weighted_ypred is wt_return

,jdate,wt_return
0,2000-01-31,-0.271680
1,2000-02-29,1.164710
2,2000-03-31,0.510828
3,2000-04-30,-0.854505
4,2000-05-31,-0.639032


In [19]:
# Using SP500
SP500_VW_ret['jdate'] = pd.to_datetime(SP500_VW_ret['jdate'])
SP500_VW_ret['Year'] = SP500_VW_ret['jdate'].dt.year
merged_Sp500 = SP500_wt_ret.merge(train_val, left_on = 'jdate', right_on='DateYM', how='left')
result_SP500 = merged_Sp500.groupby('jdate')['retadj'].sum().reset_index()

In [20]:
merged_Sp500.head()

,jdate,wt_return,AbnormalAccruals,Accruals,AccrualsBM,Activism1,Activism2,AdExp,AgeIPO,AM,...,XFIN,zerotrade,zerotradeAlt1,zerotradeAlt12,mvel1,mom1m,baspread,sic2,retadj,test_fold
0,2000-01-31,-0.044375,0.585548,0.732207,-0.421196,-0.038095,0.006024,0.000656,-0.008746,0.239803,...,0.587691,0.727203,0.755337,0.735880,-0.706514,-0.108218,-0.924625,-0.094187,-0.044118,-1.0
1,2000-01-31,-0.044375,0.000187,0.585635,-0.421196,-0.038095,0.006024,-0.640420,-0.008746,0.669079,...,0.000178,0.869787,0.856319,0.858795,-0.160283,-0.722884,-0.271618,0.202600,-0.025641,-1.0
2,2000-01-31,-0.044375,0.000187,0.431589,-0.421196,-0.038095,0.006024,-0.024934,-0.008746,0.952303,...,0.000178,0.817115,0.694479,0.844271,-0.551842,-0.878148,-0.369435,0.202600,-0.008475,-1.0
3,2000-01-31,-0.044375,0.660801,0.434514,-0.421196,-0.038095,0.006024,0.000656,-0.008746,-0.929276,...,0.072928,-0.549802,-0.921227,-0.520441,0.032935,0.407444,0.761922,-0.436473,-0.097276,-1.0
4,2000-01-31,-0.044375,-0.561213,-0.635359,-0.421196,-0.925397,0.006024,0.000656,-0.008746,-0.302303,...,0.972963,-0.091953,0.229693,-0.278913,0.297878,-0.582238,-0.101842,-0.300343,-0.099338,-1.0


In [21]:
# yearly_returns_SP500 = SP500_VW_ret.groupby('Year')['ret'].mean().reset_index()
# merged_Sp500 = yearly_returns_SP500.merge(result_SP500, on='Year', how='inner')
merged_Sp500['Sharpe_Ratio'] = (merged_Sp500['ret'] - 0.045) / merged_Sp500['wt_return']
merged_Sp500['R^2'] = (merged_Sp500['ret'] - merged_Sp500['wt_return'])**2
merged_Sp500

KeyError: 'ret'